In [ ]:
!nvidia-smi

Fri Mar 12 02:51:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    22W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Colab如何加装Google drive 硬盘：https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/'Shared drives'/JXNU_C/HXM/IPC_to_CLC/

/content/drive/Shared drives/JXNU_C/HXM/IPC_to_CLC


In [ ]:
!pwd

/content/drive/Shared drives/JXNU_C/HXM/IPC_to_CLC


In [ ]:
!ls

Bilinear_CNN.ipynb
Bi_LSTM.ipynb
data_KFold_1
drive-download-20210306T124752Z-001.zip
error_log
ipc_to_clc_data_and_result_类目相似度_结果备份
log
other_network
output
sentence_transformers
si_model.ipynb
test_sentence_transformers.ipynb
Text_CNN.ipynb
train_sentence_transformers.ipynb
Transformer_Encoder.ipynb


In [ ]:
!pip install ipdb transformers sentence-transformers

## 基于sentence_transformers实现预训练语言模型孪生网络，并进行训练。

In [ ]:
import os
import math
import logging
import ipdb
import torch

from torch import nn
from torch.utils.data import DataLoader
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, InputExample, LoggingHandler, SentenceTransformer, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
from datetime import datetime
from tqdm import tqdm

#设置随机种子，保证实验的可重复性
seed = 42

torch.manual_seed(seed)  # 为CPU设置随机种子
torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
#如果使用多个GPU，应该使用torch.cuda.manual_seed_all()为所有的GPU设置种子

#用以保证模型的可重复性，实验的可重复性
torch.backends.cudnn.deterministic = True

#设置日志格式
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])


#模型名字  distiluse-base-multilingual-cased-v2 
model_name = "bert-base-chinese"
#是否自定义模型(与模型名字有关)
custom_model = True

batch_size = 64
num_epochs = 31
weight_decay = 0.01

# 损失函数类型： sample_ContrastiveLoss   sample_MultipleNegativesRankingLoss   sample_TripletLoss
loss_type = "sample_ContrastiveLoss"

# 五折交叉验证，第几折选择: 0 1 2 3 4
kfold = "4"

#训练好的模型保存路径
model_save_path = "output/" + 'Bilinear_CNN-' + str(kfold) + "-" + str(num_epochs) + "-" + "training_ipc-to-clc_" + model_name + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

#判断GPU是否可用
device = "cuda" if torch.cuda.is_available() else 'cpu'



#自定义模型，包括transformers中的模型也可以用(可以自定义模型框架)
if custom_model:
  # Use model_name for mapping tokens to embeddings
  word_embedding_model = models.Transformer(model_name, model_args={'output_hidden_states': True})

  b_cnn_model = models.BilinearCNN(word_embedding_model.get_word_embedding_dimension(), n_filters=128, filter_sizes=[3, 4, 5], out_word_embedding_dimension=768, dropout_rate=0.0)
  #[3, 4, 5]

  model = SentenceTransformer(modules=[word_embedding_model, b_cnn_model], device=device)

#加载sentence_transformers已经训练过的模型
else:
  model = SentenceTransformer(model_name, device=device)


#train文件路径
train_dataset_path = "data_KFold_1/" + loss_type + "/data_sample_" + kfold + "/ipc_clc_train.txt"

#valid文件路径(这个只有 sample_MultipleNegativesRankingLoss 需要，因为训练数据中没有label，需要在训练前将验证数据从训练集中分离出来)
valid_dataset_path = "data_KFold_1/" + loss_type + "/data_sample_" + kfold + "/ipc_clc_valid.txt"


#读入训练数据
logging.info("读入训练数据...")

train_shuliang = 0

train_samples = []
if loss_type == "sample_MultipleNegativesRankingLoss":
  with open(train_dataset_path, 'r', encoding='utf8') as fIn:
    for row in fIn:
      train_shuliang += 1
      row = row.strip()
      row = row.split('\t')
      train_samples.append(InputExample(texts=[row[0], row[1]])) # MultipleNegativesRankingLoss 无label
else:
  with open(train_dataset_path, 'r', encoding='utf8') as fIn:
      train_valid_number = 0
      for row in fIn:
        train_valid_number = train_valid_number + 1
        if train_valid_number % 5 == 0:
          continue
        else:
          train_shuliang += 1
          row = row.strip()
          row = row.split('\t')
          if loss_type == "sample_ContrastiveLoss":
            train_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) # ContrastiveLoss 有label
          else:
            train_samples.append(InputExample(texts=[row[0], row[1], row[2]])) # TripletLoss 无label

print('训练集大小：' + str(train_shuliang))        

#构造训练数据 DataLoader
train_data = SentencesDataset(train_samples, model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)


#定义训练的损失函数
if loss_type == "sample_ContrastiveLoss":
  train_loss = losses.OnlineContrastiveLoss(model=model, margin=0.5)  #有label
elif loss_type == "sample_MultipleNegativesRankingLoss":
  train_loss = losses.MultipleNegativesRankingLoss(model=model)  #无label
else:
  train_loss = losses.TripletLoss(model=model, triplet_margin=1)  #无label 默认triplet_margin=5




#读入验证集数据
logging.info("读入验证集数据...")

valid_shuliang = 0

valid_samples = []
#sample_MultipleNegativesRankingLoss直接从文件中读取，其他loss的验证集将直接从训练数据中得到，1/5的训练集数据将作为验证集
#需要注意的是，如果只有一种label，则evaluator无法进行验证，所以是二分类的要保证验证集中两种类别都有
if loss_type == "sample_MultipleNegativesRankingLoss":
  with open(valid_dataset_path, 'r', encoding='utf8') as fIn:
    for row in fIn:
      valid_shuliang += 1
      row = row.strip()
      row = row.split('\t')
      valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) #验证集需要 label
else:   
  with open(train_dataset_path, 'r', encoding='utf8') as fIn:
    train_valid_number = 0
    for row in fIn:
      train_valid_number = train_valid_number + 1
      if train_valid_number % 5 == 0:
        valid_shuliang += 1
        row = row.strip()
        row = row.split('\t')
        if loss_type == "sample_ContrastiveLoss":
          valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(row[2]))) #验证集需要 label
        else:
          valid_samples.append(InputExample(texts=[row[0], row[1]], label=int(1))) # TripletLoss pos
          valid_samples.append(InputExample(texts=[row[0], row[2]], label=int(0))) # TripletLoss neg
      else:
        continue

print('验证集大小：' + str(valid_shuliang))  

#构造验证集数据 DataLoader
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_samples, name='ipc-clc-valid')


#warmup步数  这个也有很大的随机性
warmup_steps = math.ceil(len(train_samples) * num_epochs / batch_size * 0.15) #15% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


#模型训练
model.fit(train_objectives = [(train_dataloader, train_loss)],
          epochs = num_epochs,
          #optimizer_class = 'transformers.optimization.AdamW',
          optimizer_params = {'correct_bias': False, 'eps': 1e-06, 'lr': 2e-05},
          scheduler = 'WarmupLinear',
          evaluator = evaluator,
          evaluation_steps = 10,
          save_best_model = True,
          weight_decay = weight_decay,
          warmup_steps = warmup_steps,
          max_grad_norm = 1,
          output_path = model_save_path
         )

2021-03-12 02:51:30 - 读入训练数据...
训练集大小：1280
2021-03-12 02:51:30 - 读入验证集数据...
验证集大小：320
2021-03-12 02:51:30 - Warmup-steps: 93


2021-03-12 02:51:47 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 0 after 10 steps:
2021-03-12 02:51:48 - Cosine-Similarity :	Pearson: 0.8435	Spearman: 0.8344
2021-03-12 02:51:48 - Manhattan-Distance:	Pearson: 0.8476	Spearman: 0.8359
2021-03-12 02:51:48 - Euclidean-Distance:	Pearson: 0.8483	Spearman: 0.8354
2021-03-12 02:51:48 - Dot-Product-Similarity:	Pearson: 0.7081	Spearman: 0.7290
2021-03-12 02:51:48 - Save model to output/Bilinear_CNN-4-31-training_ipc-to-clc_bert-base-chinese-2021-03-12_02-51-27
2021-03-12 02:52:05 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 0 after 20 steps:
2021-03-12 02:52:06 - Cosine-Similarity :	Pearson: 0.8685	Spearman: 0.8428
2021-03-12 02:52:06 - Manhattan-Distance:	Pearson: 0.8826	Spearman: 0.8473
2021-03-12 02:52:06 - Euclidean-Distance:	Pearson: 0.8824	Spearman: 0.8478
2021-03-12 02:52:06 - Dot-Product-Similarity:	Pearson: 0.7841	Spearman: 0.7916
2021-03-12 02:52:06 - S

2021-03-12 02:52:25 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 1 after 10 steps:
2021-03-12 02:52:26 - Cosine-Similarity :	Pearson: 0.8585	Spearman: 0.8441
2021-03-12 02:52:26 - Manhattan-Distance:	Pearson: 0.8718	Spearman: 0.8480
2021-03-12 02:52:26 - Euclidean-Distance:	Pearson: 0.8726	Spearman: 0.8483
2021-03-12 02:52:26 - Dot-Product-Similarity:	Pearson: 0.6181	Spearman: 0.6871
2021-03-12 02:52:26 - Save model to output/Bilinear_CNN-4-31-training_ipc-to-clc_bert-base-chinese-2021-03-12_02-51-27
2021-03-12 02:52:43 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 1 after 20 steps:
2021-03-12 02:52:45 - Cosine-Similarity :	Pearson: 0.8523	Spearman: 0.8388
2021-03-12 02:52:45 - Manhattan-Distance:	Pearson: 0.8759	Spearman: 0.8432
2021-03-12 02:52:45 - Euclidean-Distance:	Pearson: 0.8758	Spearman: 0.8431
2021-03-12 02:52:45 - Dot-Product-Similarity:	Pearson: 0.7652	Spearman: 0.7882

2021-03-12 02:52:45 - 

2021-03-12 02:53:01 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 2 after 10 steps:
2021-03-12 02:53:02 - Cosine-Similarity :	Pearson: 0.8439	Spearman: 0.8465
2021-03-12 02:53:02 - Manhattan-Distance:	Pearson: 0.8799	Spearman: 0.8481
2021-03-12 02:53:02 - Euclidean-Distance:	Pearson: 0.8806	Spearman: 0.8484
2021-03-12 02:53:02 - Dot-Product-Similarity:	Pearson: 0.7085	Spearman: 0.7303
2021-03-12 02:53:02 - Save model to output/Bilinear_CNN-4-31-training_ipc-to-clc_bert-base-chinese-2021-03-12_02-51-27
2021-03-12 02:53:19 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 2 after 20 steps:
2021-03-12 02:53:20 - Cosine-Similarity :	Pearson: 0.7969	Spearman: 0.8363
2021-03-12 02:53:20 - Manhattan-Distance:	Pearson: 0.8691	Spearman: 0.8475
2021-03-12 02:53:20 - Euclidean-Distance:	Pearson: 0.8693	Spearman: 0.8476
2021-03-12 02:53:20 - Dot-Product-Similarity:	Pearson: 0.6400	Spearman: 0.6753

2021-03-12 02:53:20 - 

2021-03-12 02:53:36 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 3 after 10 steps:
2021-03-12 02:53:38 - Cosine-Similarity :	Pearson: 0.7677	Spearman: 0.8336
2021-03-12 02:53:38 - Manhattan-Distance:	Pearson: 0.8324	Spearman: 0.8346
2021-03-12 02:53:38 - Euclidean-Distance:	Pearson: 0.8341	Spearman: 0.8349
2021-03-12 02:53:38 - Dot-Product-Similarity:	Pearson: 0.6368	Spearman: 0.6683
2021-03-12 02:53:53 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 3 after 20 steps:
2021-03-12 02:53:54 - Cosine-Similarity :	Pearson: 0.8096	Spearman: 0.8378
2021-03-12 02:53:54 - Manhattan-Distance:	Pearson: 0.8492	Spearman: 0.8393
2021-03-12 02:53:54 - Euclidean-Distance:	Pearson: 0.8504	Spearman: 0.8394
2021-03-12 02:53:54 - Dot-Product-Similarity:	Pearson: 0.7390	Spearman: 0.7665

2021-03-12 02:53:54 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 3:
2021-03-12 02:53:56 - Cosine

...

2021-03-12 03:06:53 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 26 after 10 steps:
2021-03-12 03:06:55 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:06:55 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:06:55 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:06:55 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249
2021-03-12 03:07:10 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 26 after 20 steps:
2021-03-12 03:07:12 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:07:12 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:07:12 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:07:12 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249

2021-03-12 03:07:12 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 26:
2021-03-12 03:07:13 - Cos

2021-03-12 03:07:28 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 27 after 10 steps:
2021-03-12 03:07:29 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:07:29 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:07:29 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:07:29 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249
2021-03-12 03:07:45 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 27 after 20 steps:
2021-03-12 03:07:46 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:07:46 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:07:46 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:07:46 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249

2021-03-12 03:07:46 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 27:
2021-03-12 03:07:48 - Cos

2021-03-12 03:08:03 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 28 after 10 steps:
2021-03-12 03:08:04 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:08:04 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:08:04 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:08:04 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249
2021-03-12 03:08:19 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 28 after 20 steps:
2021-03-12 03:08:21 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:08:21 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:08:21 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:08:21 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249

2021-03-12 03:08:21 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 28:
2021-03-12 03:08:22 - Cos

2021-03-12 03:08:37 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 29 after 10 steps:
2021-03-12 03:08:39 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:08:39 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:08:39 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:08:39 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249
2021-03-12 03:08:54 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 29 after 20 steps:
2021-03-12 03:08:55 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:08:55 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:08:55 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:08:55 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249

2021-03-12 03:08:55 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 29:
2021-03-12 03:08:57 - Cos

2021-03-12 03:09:12 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 30 after 10 steps:
2021-03-12 03:09:13 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:09:13 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:09:13 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:09:13 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249
2021-03-12 03:09:29 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset in epoch 30 after 20 steps:
2021-03-12 03:09:30 - Cosine-Similarity :	Pearson: 0.8184	Spearman: 0.8415
2021-03-12 03:09:30 - Manhattan-Distance:	Pearson: 0.8683	Spearman: 0.8442
2021-03-12 03:09:30 - Euclidean-Distance:	Pearson: 0.8690	Spearman: 0.8446
2021-03-12 03:09:30 - Dot-Product-Similarity:	Pearson: 0.7017	Spearman: 0.7249

2021-03-12 03:09:30 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset after epoch 30:
2021-03-12 03:09:31 - Cos

In [ ]:
#利用训练好的模型得到测试集结果
model = SentenceTransformer(model_save_path)

evaluator(model, output_path=model_save_path)

2021-03-12 03:09:31 - Load pretrained SentenceTransformer: output/Bilinear_CNN-4-31-training_ipc-to-clc_bert-base-chinese-2021-03-12_02-51-27
2021-03-12 03:09:31 - Load SentenceTransformer from folder: output/Bilinear_CNN-4-31-training_ipc-to-clc_bert-base-chinese-2021-03-12_02-51-27
2021-03-12 03:09:34 - Use pytorch device: cuda
2021-03-12 03:09:34 - EmbeddingSimilarityEvaluator: Evaluating the model on ipc-clc-valid dataset:
2021-03-12 03:09:36 - Cosine-Similarity :	Pearson: 0.8439	Spearman: 0.8465
2021-03-12 03:09:36 - Manhattan-Distance:	Pearson: 0.8799	Spearman: 0.8481
2021-03-12 03:09:36 - Euclidean-Distance:	Pearson: 0.8806	Spearman: 0.8484
2021-03-12 03:09:36 - Dot-Product-Similarity:	Pearson: 0.7085	Spearman: 0.7303


0.848370746980798

## 利用预训练语言模型进行验证（这里只需要测试数据，k折交叉验证可以直接改变测试数据）

In [ ]:
#可选，如果不进行上面的训练，则这里不注释，第0个文件中保存有全部1000条数据
#import torch
#from sentence_transformers import SentenceTransformer, util
#from tqdm import tqdm
#kfold = "0"

#读入IPC和CLC测试数据
read_ipc_file = open("data_KFold_1/" + "origin/" + "data_origin_" + kfold + "/ipc_valid.txt", 'r', encoding='utf-8')
read_clc_file = open("data_KFold_1/" + "origin/" + "data_origin_" + kfold + "/clc_valid.txt", 'r', encoding='utf-8')

#存储IPC和CLC的分别类号(key)和匹配字符串(value)
ipc_key_list = []
ipc_value_list = []

clc_key_list = []
clc_value_list = []

for line in read_ipc_file:
    line = line.strip()
    line = line.split(' ')
    assert len(line) == 2, 'IPC.txt文件中只应该有两列，每一列用一个空格隔开！' + str(line[0])
    ipc_key_list.append(str(line[0]))
    ipc_value_list.append(str(line[1]))
    
        
for line in read_clc_file:
    line = line.strip()
    line = line.split(' ')
    assert len(line) == 2, 'CLC.txt文件中只应该有两列，每一列用一个空格隔开！' + str(line[0])
    clc_key_list.append(str(line[0]))
    clc_value_list.append(str(line[1]))
    
assert len(ipc_key_list) == len(clc_key_list), 'IPC.txt文件和CLC.txt文件要一一对应，行数要一致！'


match_clc_key_dict = dict()

for ipc_key_i, target in enumerate(ipc_key_list):
    match_clc_key_dict[str(ipc_key_i)] = []
    for ipc_key_j, every_ipc_key in enumerate(ipc_key_list):
        if every_ipc_key == target:
            match_clc_key_dict[str(ipc_key_i)].append(clc_key_list[ipc_key_j])

#用于计算IPC每一类别包含的数量
ipc_section_number_dict = {
    'A' : 0,
    'B' : 0,
    'C' : 0,
    'D' : 0,
    'E' : 0,
    'F' : 0,
    'G' : 0,
    'H' : 0
}
for ipc_key_i in ipc_key_list:
  ipc_section_number_dict[str(ipc_key_i[0])] += 1



"""
思路：
利用预训练语言模型对IPC和CLC字符串进行编码，得到句子级别的语境向量，
之后对两者进行语义相似度计算（余弦相似度）。

步骤：
1.我们将IPC作为查询字符串，CLC作为待匹配的字符串。

2.一条IPC字符串需要与每一条CLC字符串进行语义相似度计算（余弦相似度）。

3.计算完毕后，取出相似度最高的前n条CLC分类号，作为与当前IPC形成映射的CLC分类号。

4.计算准确率，TopK准确率。
"""

#判断GPU是否可用
device = "cuda" if torch.cuda.is_available() else 'cpu'
#模型路径 （optional:  model_save_path   "bert-base-chinese"   "distiluse-base-multilingual-cased-v2"  "hfl/chinese-bert-wwm"）
model_path = model_save_path

#加载预训练语言模型（可更换,传入模型名称/模型路径）
model = SentenceTransformer(model_path, device=device)

#将CLC查询字符串转化为语境向量，并转化为tensor
clc_corpus_embeddings = model.encode(clc_value_list, convert_to_tensor=True, device=device)

'''
ipdb> len(clc_value_list)
200

ipdb> clc_corpus_embeddings.shape
torch.Size([200, 768])
'''

#可规定TopK准确率
top_k = 10

#存储相似度最高的前n条CLC分类号
clc_top_k_results = dict()

#存储匹配正确的数量(只取匹配度最高的)
correct = 0

#存储TopK中匹配正确的数量
top_k_correct = 0

#存储top2-TopK中匹配正确的数量(包含了topK)
top2_to_topk_correct_dict = {
    '2' : 0,
    '3' : 0,
    '4' : 0,
    '5' : 0,
    '6' : 0,
    '7' : 0,
    '8' : 0,
    '9' : 0,
    '10' : 0
}

#存储每一个IPC类别的正确个数
ipc_section_correct_dict = {
    'A' : 0,
    'B' : 0,
    'C' : 0,
    'D' : 0,
    'E' : 0,
    'F' : 0,
    'G' : 0,
    'H' : 0
}

#存储相似度最高匹配错误的ipc和clc句对
error_match_ipc_clc_pair = []

#存储topk匹配错误的ipc和clc句对
topk_error_match_ipc_clc_pair = []

pbar = tqdm(ipc_value_list)
for ipc_i, ipc_query in enumerate(pbar):
    
    #将IPC查询字符串转化为语境向量，并转化为tensor
    ipc_query_embedding = model.encode(ipc_query, convert_to_tensor=True, device=device)
    '''
    ipdb> ipc_query
    '鞋类;鞋类的特征;鞋类的部件;鞋帮;靴腿;增强件;其他单个鞋部件;鞋帮;靴腿;'

    ipdb> ipc_query_embedding .shape
    torch.Size([768])
    '''
    #计算两者的语义相似度计算（余弦相似度）
    cos_scores = util.pytorch_cos_sim(ipc_query_embedding, clc_corpus_embeddings)[0]
    '''
    ipdb> ipc_query_embedding.shape
    torch.Size([768])

    ipdb> clc_corpus_embeddings.shape
    torch.Size([200, 768])

    ipdb> cos_scores .shape
    torch.Size([200])
    '''
    cos_scores = cos_scores.cpu()
    

    #使用 torch.topk 找到相似度最高的前n条CLC分类号
    top_results = torch.topk(cos_scores, k=top_k)
    '''
    ipdb> top_results 
    torch.return_types.topk(
    values=tensor([0.9510, 0.8601, 0.8227, 0.8227, 0.8227, 0.7801, 0.7547, 0.7545, 0.7543,
            0.7527]),
    indices=tensor([  0,  11, 118, 126, 119,  36,  47,  23,  90,  74]))
    '''
    clc_top_k_results[str(ipc_i)] = []
    to_match_clc_key_list = []
    
    #只有前K个结果
    for score, idx in zip(top_results[0], top_results[1]):
        clc_top_k_results[str(ipc_i)].append((str(clc_key_list[idx]), str(clc_value_list[idx]), score))
        to_match_clc_key_list.append(str(clc_key_list[idx]))
    
    #匹配正确的数量(只取第一个匹配度最高的)
    if clc_top_k_results[str(ipc_i)][0][0] in match_clc_key_dict[str(ipc_i)]:
        correct = correct + 1
        ipc_section_correct_dict[str(ipc_key_list[ipc_i][0])] += 1
    else:
        error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                        '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                        str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                        str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                        str(clc_top_k_results[str(ipc_i)][0][1]))
    
    #TopK中匹配正确的数量(10个的)
    if len(match_clc_key_dict[str(ipc_i)]) == 1:
      assert match_clc_key_dict[str(ipc_i)][0] == str(clc_key_list[ipc_i]), 'TopK匹配有问题！--1'
      if str(clc_key_list[ipc_i]) in to_match_clc_key_list:
        top_k_correct = top_k_correct + 1
      else:
        topk_error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                          '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                          str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                          str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                          str(clc_top_k_results[str(ipc_i)][0][1]))
    elif len(match_clc_key_dict[str(ipc_i)]) > 1:
      for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
        if str(each_clc_key_1) in to_match_clc_key_list:
          top_k_correct = top_k_correct + 1
          break
        else:
          topk_error_match_ipc_clc_pair.append(str(ipc_key_list[ipc_i]) + '  ' + str(ipc_value_list[ipc_i]) + \
                                            '\t|||\t' + str(clc_key_list[ipc_i]) + '  ' + \
                                            str(clc_value_list[ipc_i]) + '\t|||\t' + \
                                            str(clc_top_k_results[str(ipc_i)][0][0]) + '  ' + \
                                            str(clc_top_k_results[str(ipc_i)][0][1]))
    else:
      print('TopK匹配有问题！--2')

    
    #计算tok2-topK每一个的准确率
    for topk_index in range(1, top_k):
      for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
        if str(each_clc_key_1) in to_match_clc_key_list[:topk_index+1]:
          top2_to_topk_correct_dict[str(topk_index+1)] += 1
          break

    pbar.set_description("Processing %s" % str(ipc_i+1))
    
    
#计算准确率，TopK准确率
ipc_count_number = len(ipc_key_list)
acc = correct/ipc_count_number
top_k_acc = top_k_correct/ipc_count_number
print('\nFinal acc: {:.2f}% .'.format(acc*100))
print('\nFinal top_{}_acc: {:.2f}% .'.format(top_k, top_k_acc*100))
print('\n--------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
  print('\nFinal top_{}_acc: {:.2f}% .'.format(topk_name, (top_correct_num/ipc_count_number)*100))

2021-03-12 03:09:36 - Load pretrained SentenceTransformer: output/Bilinear_CNN-4-31-training_ipc-to-clc_bert-base-chinese-2021-03-12_02-51-27
2021-03-12 03:09:36 - Load SentenceTransformer from folder: output/Bilinear_CNN-4-31-training_ipc-to-clc_bert-base-chinese-2021-03-12_02-51-27


  0%|          | 0/200 [00:00<?, ?it/s]

Processing 1:   0%|          | 0/200 [00:00<?, ?it/s]

Processing 2:   1%|          | 2/200 [00:00<00:14, 13.74it/s]

Processing 3:   1%|          | 2/200 [00:00<00:14, 13.74it/s]

...

Processing 198:  99%|█████████▉| 198/200 [00:19<00:00,  7.68it/s]

Processing 199: 100%|█████████▉| 199/200 [00:19<00:00,  8.19it/s]

Processing 200: 100%|█████████▉| 199/200 [00:19<00:00,  8.19it/s]

Processing 200: 100%|██████████| 200/200 [00:19<00:00, 10.12it/s]


Final acc: 76.00% .

Final top_10_acc: 98.50% .

--------------------


Final top_2_acc: 86.00% .

Final top_3_acc: 93.50% .

Final top_4_acc: 95.50% .

Final top_5_acc: 96.50% .

Final top_6_acc: 97.00% .

Final top_7_acc: 97.50% .

Final top_8_acc: 98.00% .

Final top_9_acc: 98.50% .

Final top_10_acc: 98.50% .


In [ ]:
#结果统计
print('总数：{}\ntop1 正确个数：{}\ntop1 错误个数：{}\n'.format(ipc_count_number, correct, len(error_match_ipc_clc_pair)))

print('总数：{}\ntop10 正确个数：{}\ntop10 错误个数：{}\n'.format(ipc_count_number, top_k_correct, len(topk_error_match_ipc_clc_pair)))


print('\n---------------------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
  print('总数：{}\ntop{} 正确个数：{}\ntop{} 错误个数：{}\n'.format(ipc_count_number, topk_name, top_correct_num, topk_name, ipc_count_number-top_correct_num))

总数：200
top1 正确个数：152
top1 错误个数：48

总数：200
top10 正确个数：197
top10 错误个数：3


---------------------------------

总数：200
top2 正确个数：172
top2 错误个数：28

总数：200
top3 正确个数：187
top3 错误个数：13

总数：200
top4 正确个数：191
top4 错误个数：9

总数：200
top5 正确个数：193
top5 错误个数：7

总数：200
top6 正确个数：194
top6 错误个数：6

总数：200
top7 正确个数：195
top7 错误个数：5

总数：200
top8 正确个数：196
top8 错误个数：4

总数：200
top9 正确个数：197
top9 错误个数：3

总数：200
top10 正确个数：197
top10 错误个数：3



In [ ]:
#计算每一个IPC类别的正确个数和正确率（只有top1的）
for sec_i, sec_cor in ipc_section_correct_dict.items():
  print('类别：{}，总数为：{}，正确个数为：{}，错误个数为：{}，正确率为：{:.2f}%\n'.format(sec_i, ipc_section_number_dict[sec_i], sec_cor, ipc_section_number_dict[sec_i]-sec_cor, (sec_cor/ipc_section_number_dict[sec_i])*100))

类别：A，总数为：18，正确个数为：16，错误个数为：2，正确率为：88.89%

类别：B，总数为：40，正确个数为：28，错误个数为：12，正确率为：70.00%

类别：C，总数为：28，正确个数为：21，错误个数为：7，正确率为：75.00%

类别：D，总数为：19，正确个数为：14，错误个数为：5，正确率为：73.68%

类别：E，总数为：18，正确个数为：14，错误个数为：4，正确率为：77.78%

类别：F，总数为：27，正确个数为：22，错误个数为：5，正确率为：81.48%

类别：G，总数为：24，正确个数为：18，错误个数为：6，正确率为：75.00%

类别：H，总数为：26，正确个数为：19，错误个数为：7，正确率为：73.08%



In [ ]:
#可选，如果不进行上面的训练，则这里不注释
loss_type = 'Bilinear_CNN'
#kfold = model_path
#num_epochs = 'no_train'

#相似度最高的错误匹配（topk=1）
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'top1_error_text_log.txt', 'w', encoding='utf-8')

for k_v in error_match_ipc_clc_pair:
  f_write.write(str(k_v) + '\n')

f_write.close()

In [ ]:
#topk的错误匹配结果
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'topk_error_text_log.txt', 'w', encoding='utf-8')

for k_v in topk_error_match_ipc_clc_pair:
  f_write.write(str(k_v) + '\n')

f_write.close()

In [ ]:
#topk的匹配结果
f_write = open('error_log/' + loss_type + '_log/' + 'data_origin_' + kfold + '/' + str(num_epochs) + '-' + str(round(acc*100, 2)) + '%-' + 'topk_error_score_log.txt', 'w', encoding='utf-8')

for k_index, v_list_tuple in clc_top_k_results.items():
  for each_tuple in v_list_tuple:
    f_write.write(ipc_key_list[int(k_index)] + '----' + each_tuple[0] + each_tuple[1] + str(each_tuple[2].item()) + '\n')

  f_write.write('------------------------------------------------------------------------------' + '\n')

f_write.close()